<h2>About this Project</h2>
<p>In this project, you will implement cross validation to pick the best depth (hyperparameter) for a regression tree, again using the ION data set.</p>

<h3>Evaluation</h3>

<p><strong>This project must be successfully completed and submitted in order to receive credit for this course. Your score on this project will be included in your final grade calculation.</strong><p>
    
<p>You are expected to write code where you see <em># YOUR CODE HERE</em> within the cells of this notebook. Not all cells will be graded; code input cells followed by cells marked with <em>#Autograder test cell</em> will be graded. Upon submitting your work, the code you write at these designated positions will be assessed using an "autograder" that will run all test cells to assess your code. You will receive feedback from the autograder that will identify any errors in your code. Use this feedback to improve your code if you need to resubmit. Be sure not to change the names of any provided functions, classes, or variables within the existing code cells, as this will interfere with the autograder. Also, remember to execute all code cells sequentially, not just those you’ve edited, to ensure your code runs properly.</p>
    
<p>You can resubmit your work as many times as necessary before the submission deadline. If you experience difficulty or have questions about this exercise, use the Q&A discussion board to engage with your peers or seek assistance from the instructor.<p>

<p>Before starting your work, please review <a href="https://s3.amazonaws.com/ecornell/global/eCornellPlagiarismPolicy.pdf">eCornell's policy regarding plagiarism</a> (the presentation of someone else's work as your own without source credit).</p>

<h3>Submit Code for Autograder Feedback</h3>

<p>Once you have completed your work on this notebook, you will submit your code for autograder review. Follow these steps:</p>

<ol>
  <li><strong>Save your notebook.</strong></li>
  <li><strong>Mark as Completed —</strong> In the blue menu bar along the top of this code exercise window, you’ll see a menu item called <strong>Education</strong>. In the <strong>Education</strong> menu, click <strong>Mark as Completed</strong> to submit your code for autograder/instructor review. This process will take a moment and a progress bar will show you the status of your submission.</li>
	<li><strong>Review your results —</strong> Once your work is marked as complete, the results of the autograder will automatically be presented in a new tab within the code exercise window. You can click on the assessment name in this feedback window to see more details regarding specific feedback/errors in your code submission.</li>
  <li><strong>Repeat, if necessary —</strong> The Jupyter notebook will always remain accessible in the first tabbed window of the exercise. To reattempt the work, you will first need to click <strong>Mark as Uncompleted</strong> in the <strong>Education</strong> menu and then proceed to make edits to the notebook. Once you are ready to resubmit, follow steps one through three. You can repeat this procedure as many times as necessary.</li>
</ol>
<p>You can also download a copy of this notebook in multiple formats using the <strong>Download as</strong> option in the <strong>File</strong> menu above.</p>

## Get Started

<p>Let's import a few packages that you will need. You will work with the <a href="https://archive.ics.uci.edu/ml/datasets/Ionosphere">ION</a> dataset for this project.</p> 

In [1]:
import numpy as np
from pylab import *
from numpy.matlib import repmat
import matplotlib.pyplot as plt
from scipy.io import loadmat
import time

%matplotlib notebook

from helper import *

print('You\'re running python %s' % sys.version.split(' ')[0])

You're running python 3.6.8


In [2]:
data = loadmat("ion.mat")
xTr  = data['xTr'].T
yTr  = data['yTr'].flatten()
xTe  = data['xTe'].T
yTe  = data['yTe'].flatten()

We also developed a regression tree classifier for you to use. In addition to what you implemented in the first project, we also added a <code>depth</code> argument to the classifier. This <code>depth</code> argument allows us to restrict the depth of the tree model. 

The following code cell shows you how to instantiate a regression tree.

In [3]:
# Create a regression tree with no restriction on its depth. 
# This is equivalent to what you implemented in the previous project
# if you want to create a tree of max depth k
# then call h.RegressionTree(depth=k)
tree = RegressionTree(depth=np.inf)

# To fit/train the regression tree
tree.fit(xTr, yTr)

# To use the trained regression tree to make prediction
pred = tree.predict(xTr)

We have also created a square loss function that takes in the prediction <code>pred</code> and ground truth <code>truth</code> and returns the average square loss between prediction and ground truth. 

In [4]:
def square_loss(pred, truth):
    return np.mean((pred - truth)**2)

Now, look at the performance of your tree on both the training set and test set using the code cell below.

In [5]:
print('Training Loss: {:.4f}'.format(square_loss(tree.predict(xTr), yTr)))
print('Test Loss: {:.4f}'.format(square_loss(tree.predict(xTe), yTe)))

Training Loss: 0.0000
Test Loss: 0.6857


## Implement Cross Validation

As you can see, your tree achives zero training loss on the training set but not zero test loss. Clearly, the model is overfitting! To reduce overfitting, you need to control the depth of the tree. One way to pick the optimal depth is to do kFold Cross Validation. To do so, you will first implement <code>grid_search</code>, which finds the best depths given a training set and validation set. Then you will implement <code>generate_kFold</code>, which generates the split for kFold cross validation. Finally, you will combine the two functions to implement <code>cross_validation</code>.

<h3>Part One: Implement <code>grid_search</code>[Graded]</h3>

Implement the function <code>grid_search</code>, which takes in a training set <code>xTr, yTr</code>, a validation set <code>xVal, yVal</code> and a list of tree depth candidates <code>depths</code>. Your job here is to fit a regression tree for each depth candidate on the training set <code>xTr, yTr</code>, evaluate the fitted tree on the validation set <code>xVal, yVal</code> and then pick the candidate that yields the lowest loss for the validation set. Note: in the event of a tie, return the smallest depth candidate.

In [6]:
def grid_search(xTr, yTr, xVal, yVal, depths):
    '''
    Input:
        xTr: nxd matrix
        yTr: n vector
        xVal: mxd matrix
        yVal: m vector
        depths: a list of len k
    Return:
        best_depth: the depth that yields that lowest loss on the validation set
        training losses: a list of len k. the i-th entry corresponds to the the training loss
                the tree of depths[i]
        validation_losses: a list of len k. the i-th entry corresponds to the the validation loss
                the tree of depths[i]
    '''
    training_losses = []
    validation_losses = []
    best_depth = None
    
    # YOUR CODE HERE
    
    for k in depths:
        tree = RegressionTree(k)

        # To fit/train the regression tree
        tree.fit(xTr, yTr)

        # To use the trained regression tree to make prediction
        
        train_loss = square_loss(tree.predict(xTr), yTr)
        val_loss = square_loss(tree.predict(xVal), yVal)
        training_losses.append(train_loss)
        validation_losses.append(val_loss)
        
        best_depth = depths[np.argmin(validation_losses)]
    
    return best_depth, training_losses, validation_losses

In [7]:
# The following tests check that your implementation of grid search returns the correct number of training and validation loss values and the correct best depth

depths = [1,2,3,4,5]
k = len(depths)
best_depth, training_losses, validation_losses = grid_search(xTr, yTr, xTe, yTe, depths)
best_depth_grader, training_losses_grader, validation_losses_grader = grid_search_grader(xTr, yTr, xTe, yTe, depths)

# Check the length of the training loss
def grid_search_test1():
    return (len(training_losses) == k) 

# Check the length of the validation loss
def grid_search_test2():
    return (len(validation_losses) == k)

# Check the argmin
def grid_search_test3():
    return (best_depth == depths[np.argmin(validation_losses)])

def grid_search_test4():
    return (best_depth == best_depth_grader)

def grid_search_test5():
    return np.linalg.norm(np.array(training_losses) - np.array(training_losses_grader)) < 1e-7

def grid_search_test6():
    return np.linalg.norm(np.array(validation_losses) - np.array(validation_losses_grader)) < 1e-7

runtest(grid_search_test1, 'grid_search_test1')
runtest(grid_search_test2, 'grid_search_test2')
runtest(grid_search_test3, 'grid_search_test3')
runtest(grid_search_test4, 'grid_search_test4')
runtest(grid_search_test5, 'grid_search_test5')
runtest(grid_search_test6, 'grid_search_test6')

Running Test: grid_search_test1 ... ✔ Passed!
Running Test: grid_search_test2 ... ✔ Passed!
Running Test: grid_search_test3 ... ✔ Passed!
Running Test: grid_search_test4 ... ✔ Passed!
Running Test: grid_search_test5 ... ✔ Passed!
Running Test: grid_search_test6 ... ✔ Passed!


In [8]:
# Autograder test cell - worth 1 point
# runs grid search test#

In [9]:
# Autograder test cell - worth 1 point
# runs grid search test2

In [10]:
# Autograder test cell - worth 1 point
# runs grid search test3

In [11]:
# Autograder test cell - worth 1 point
# runs grid search test4

In [12]:
# Autograder test cell - worth 1 point
# runs grid search test5

In [13]:
# Autograder test cell - worth 1 point
# runs grid search test6

<h3>Part Two: Implement <code>generate_kFold</code>[Graded]</h3>

Now, implement the <code>generate_kFold</code> function, which takes in the number of training examples <code>n</code> and the number of folds <code>k</code> and returns a list of <code>k</code> folds where each fold takes the form <code>(training indices, validation indices)</code>.

For instance, if n = 3 and k = 3, then we have three indices <code>[0,1,2]</code> and we are trying to split it k=3 times to obtain different training/validation splits. 
One possible output of the the function is <code>[([0, 1], [2]), ([1, 2], [0]), ([0, 2], [1])]</code> 

In [14]:
def generate_kFold(n, k):
    '''
    Input:
        n: number of training examples
        k: number of folds
    Returns:
        kfold_indices: a list of len k. Each entry takes the form
        (training indices, validation indices)
    '''
    assert k >= 2
    kfold_indices = []
    
    # YOUR CODE HERE
    fold = n // k
    ind = np.arange(n)
    index = []
    
    for i in range(0,k-1):
        index.append(ind[fold*i:fold*(i+1)])
    index.append(ind[(k-1)*fold:])
    
    for i in range(k):
        kfold_indices.append((np.concatenate([index[j] for j in range(k) if j != i]), index[i]))
    
    return kfold_indices

generate_kFold(3,3)

[(array([1, 2]), array([0])),
 (array([0, 2]), array([1])),
 (array([0, 1]), array([2]))]

In [15]:
# The following tests check that your generate_kFold function 
# returns the correct number of total indices, 
# train and test indices, and the correct ratio

kfold_indices = generate_kFold(1004, 5)

def generate_kFold_test1():
    return len(kfold_indices) == 5 # you should generate 5 folds

def generate_kFold_test2():
    t = [((len(train_indices) + len(test_indices)) == 1004) 
         for (train_indices, test_indices) in kfold_indices]
    return np.all(t) # make sure that both for each fold, the number of examples sum up to 1004

def generate_kFold_test3():
    ratio_test = []
    for (train_indices, validation_indices) in kfold_indices:
        ratio = len(validation_indices) / len(train_indices)
        ratio_test.append((ratio > 0.24 and ratio < 0.26))
    # make sure that both for each fold, the training to validation 
    # examples ratio is in between 0.24 and 0.25
    return np.all(ratio_test) 

def generate_kFold_test4():
    train_indices_set = set() # to keep track of training indices for each fold
    validation_indices_set = set() # to keep track of validation indices for each fold
    for (train_indices, validation_indices) in kfold_indices:
        train_indices_set = train_indices_set.union(set(train_indices))
        validation_indices_set = validation_indices_set.union(set(validation_indices))
    
    # Make sure that you use all the examples in all the training fold and validation fold
    return train_indices_set == set(np.arange(1004)) and validation_indices_set == set(np.arange(1004))


runtest(generate_kFold_test1, 'generate_kFold_test1')
runtest(generate_kFold_test2, 'generate_kFold_test2')
runtest(generate_kFold_test3, 'generate_kFold_test3')
runtest(generate_kFold_test4, 'generate_kFold_test4')

Running Test: generate_kFold_test1 ... ✔ Passed!
Running Test: generate_kFold_test2 ... ✔ Passed!
Running Test: generate_kFold_test3 ... ✔ Passed!
Running Test: generate_kFold_test4 ... ✔ Passed!


In [16]:
# Autograder test cell - worth 1 point
# runs generate Kfold test1

In [17]:
# Autograder test cell - worth 1 point
# runs generate Kfold test2

In [18]:
# Autograder test cell - worth 1 point
# runs generate Kfold test3

In [19]:
# Autograder test cell - worth 1 point
# runs generate Kfold test4

<h3>Part Three: Implement <code>cross_validation</code>[Graded]</h3>

Use <code>grid_search</code> to implement the <code>cross_validation</code> function that takes in the training set <code>xTr, yTr</code>, a list of depth candidates <code>depths</code> and performs K-fold cross validation on the training set. We use <code>generate_kFold</code> to generate the K training/validation split. Using <code>indices</code>, the function will do a grid search  on each fold and return the parameter that yields the best average validation loss across the folds. Note that in event of tie, the function should return the smallest depth candidate.

In [20]:
def cross_validation(xTr, yTr, depths, indices):
    '''
    Input:
        xTr: nxd matrix (training data)
        yTr: n vector (training data)
        depths: a list (of length l) depths to be tried out
        indices: indices from generate_kFold
    Returns:
        best_depth: the best parameter 
        training losses: a list of lenth l. the i-th entry corresponds to the the average training loss
                the tree of depths[i]
        validation_losses: a list of length l. the i-th entry corresponds to the the average validation loss
                the tree of depths[i] 
    '''
    training_losses = []
    validation_losses = []
    best_depth = None
    
    # YOUR CODE HERE
    testloss = []
    valloss = []
    
    for train, val in indices:
        best_depth, training_losses, validation_losses = grid_search(xTr[train], yTr[train], xTr[val], yTr[val], depths)
        
        testloss.append(training_losses)
        valloss.append(validation_losses)
    avg_testloss = np.mean(testloss, axis=0)
    avg_valloss = np.mean(valloss, axis=0)
    best_depth = depths[np.argmin(avg_valloss)]
    
    return best_depth, avg_testloss, avg_valloss

In [21]:
# The following tests check that your implementation of cross_validation returns the correct number of training and validation losses, the correct "best depth" and the correct values for training and validation loss

depths = [1,2,3,4]
k = len(depths)

# generate indices
# the same indices will be used to cross check your solution and ours
indices = generate_kFold(len(xTr), 5)
best_depth, training_losses, validation_losses = cross_validation(xTr, yTr, depths, indices)
best_depth_grader, training_losses_grader, validation_losses_grader = cross_validation_grader(xTr, yTr, depths, indices)

# Check the length of the training loss
def cross_validation_test1():
    return (len(training_losses) == k) 

# Check the length of the validation loss
def cross_validation_test2():
    return (len(validation_losses) == k)

# Check the argmin
def cross_validation_test3():
    return (best_depth == depths[np.argmin(validation_losses)])

def cross_validation_test4():
    return (best_depth == best_depth_grader)

def cross_validation_test5():
    return np.linalg.norm(np.array(training_losses) - np.array(training_losses_grader)) < 1e-7

def cross_validation_test6():
    return np.linalg.norm(np.array(validation_losses) - np.array(validation_losses_grader)) < 1e-7

runtest(cross_validation_test1, 'cross_validation_test1')
runtest(cross_validation_test2, 'cross_validation_test2')
runtest(cross_validation_test3, 'cross_validation_test3')
runtest(cross_validation_test4, 'cross_validation_test4')
runtest(cross_validation_test5, 'cross_validation_test5')
runtest(cross_validation_test6, 'cross_validation_test6')

Running Test: cross_validation_test1 ... ✔ Passed!
Running Test: cross_validation_test2 ... ✔ Passed!
Running Test: cross_validation_test3 ... ✔ Passed!
Running Test: cross_validation_test4 ... ✔ Passed!
Running Test: cross_validation_test5 ... ✔ Passed!
Running Test: cross_validation_test6 ... ✔ Passed!


In [22]:
# Autograder test cell - worth 1 point
# runs cross validation test1

In [23]:
# Autograder test cell - worth 1 point
# runs cross validation test2

In [24]:
# Autograder test cell - worth 1 point
# runs cross validation test3

In [25]:
# Autograder test cell - worth 1 point
# runs cross validation test4

In [26]:
# Autograder test cell - worth 1 point
# runs cross validation test5

In [27]:
# Autograder test cell - worth 1 point
# runs cross validation test6